# Exploring the Datasets
*Authors: Angelika Shastapalava, Excel Espina, David Hadaller, Sam Mundle*  

### What are we using:  
1) The "Discovery" API is MTA's official developer resource to get real-time data from their NYC Bus Time service. You can get more information <a href="http://bustime.mta.info/wiki/Developers/Index">here</a>  
2) Kaggle's NYC Bus Data <a href="https://www.kaggle.com/stoney71/new-york-city-transport-statistics">here</a>

### How are we using it:
Using regression and classification techniques learned in class, we want to explore the following:  
> 1. Based on a ~10 stops/lines how closely do the actual stop times reflect the posted bus
schedules and what is the distribution around the scheduled time that busses actually
arrive?
>2. What environmental factors impact a buses schedule? What impact does time of day,
temperature, and weather have?
>3. What socioeconomic factors play into a buses schedule? Do we see better or worse
availability in neighborhoods with different average incomes?
>4. Predicting with a defined degree certainty if a bus is coming within a given time frame

### Sections:
1) [Loading the Datasets](#Loading-the-Datasets)  
2) [Cleaning the Data](#Cleaning-the-Data)  
3) [Visualizing the Data](#Visualizing-the-Data)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime

### Loading the Datasets 
We want to work with the Kaggle dataset so head over <a href="https://www.kaggle.com/stoney71/new-york-city-transport-statistics">here</a> and download the zip file. (A word of caution: the dataset is approx **5GB** when extracted!)

After you extract the data, we want to load a csv on our notebook.  

The `error_bad_lines=False` parameter fixes some formatting issues when we load in our dataset.

In [2]:
%%capture
mta = pd.read_csv('mta_1708.csv', error_bad_lines=False)
mta.set_index('PublishedLineName', inplace=True)

In [3]:
mta.head()

,RecordedAtTime,DirectionRef,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime
PublishedLineName,,,,,,,,,,,,,,,,
Q32,2017-08-01 00:01:03,0,W 32 ST/7 AV,40.749405,-73.991020,JACKSON HTS NORTHERN - 81 via ROOSVLT,40.755322,-73.886139,NYCT_7424,40.749403,-73.990841,W 32 ST/AV OF THE AMERICAS,< 1 stop away,220.0,2017-08-01 00:01:37,24:01:11
B35,2017-08-01 00:00:52,0,39 ST/1 AV,40.656456,-74.012245,BROWNSVILLE M GASTON BL via CHURCH,40.656345,-73.907188,NYCT_406,40.651330,-73.938960,CHURCH AV/E 42 ST,approaching,107.0,2017-08-01 00:02:00,23:56:12
Q83,2017-08-01 00:01:18,1,227 ST/113 DR,40.702263,-73.730339,JAMAICA HILLSIDE - 153 via LIBERTY,40.706795,-73.804100,NYCT_6449,40.706532,-73.804177,153 ST/HILLSIDE AV,at stop,25.0,2017-08-01 00:01:27,24:00:00
M60-SBS,2017-08-01 00:01:05,0,BROADWAY/W 106 ST,40.801819,-73.967644,SELECT BUS SERVICE LA GUARDIA AIRPORT,40.768074,-73.862091,NYCT_5846,40.770403,-73.917687,HOYT AV/31 ST,4.1 miles away,6519.0,2017-08-01 00:06:47,23:39:14
M60-SBS,2017-08-01 00:01:05,0,BROADWAY/W 106 ST,40.801819,-73.967644,SELECT BUS SERVICE LA GUARDIA AIRPORT,40.768074,-73.862091,NYCT_5846,40.770403,-73.917687,HOYT AV/31 ST,4.1 miles away,6519.0,2017-08-01 00:06:47,23:44:32


In [4]:
mta.dtypes

RecordedAtTime                object
DirectionRef                   int64
OriginName                    object
OriginLat                    float64
OriginLong                   float64
DestinationName               object
DestinationLat               float64
DestinationLong              float64
VehicleRef                    object
VehicleLocation.Latitude     float64
VehicleLocation.Longitude    float64
NextStopPointName             object
ArrivalProximityText          object
DistanceFromStop             float64
ExpectedArrivalTime           object
ScheduledArrivalTime          object
dtype: object

## Cleaning the Data
Looking at the head samples, we see `ExpectedArrivalTime` and `ScheduledArrivalTime` are objects of differing formats. We should fix that.

In this case, we want to compare the time difference between Expected and Schedule times when the distance from the stop is <= 30ft.

First things first: convert ```ExpectedArrivalTime```'s Timestamp format to just the time itself.

Then we need to adjust the ```ScheduledArrivalTime``` to match python's time range of 0-23 hours. Here we pass ```errors='coerce'``` parameter to convert troublesome or out of range times to NaT (Not a Time).

In [5]:
### Removing Date from the ExpectedArrivalTime

# Convert object to datetime64 
mta['expected_time'] = pd.to_datetime(mta['ExpectedArrivalTime']).dt.time
mta['scheduled_time'] = pd.to_datetime(mta['ScheduledArrivalTime'],format='%H:%M:%S', errors='coerce').dt.time
# mta['scheduled_time'] = pd.to_datetime(mta['scheduled_time'])
# temp = pd.to_datetime(mta['scheduled_time']).dt.time
# datetime.datetime.strptime().time()
mta.infer_objects().dtypes

RecordedAtTime                object
DirectionRef                   int64
OriginName                    object
OriginLat                    float64
OriginLong                   float64
DestinationName               object
DestinationLat               float64
DestinationLong              float64
VehicleRef                    object
VehicleLocation.Latitude     float64
VehicleLocation.Longitude    float64
NextStopPointName             object
ArrivalProximityText          object
DistanceFromStop             float64
ExpectedArrivalTime           object
ScheduledArrivalTime          object
expected_time                 object
scheduled_time                object
dtype: object

In [6]:
mta.head()

,RecordedAtTime,DirectionRef,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,expected_time,scheduled_time
PublishedLineName,,,,,,,,,,,,,,,,,,
Q32,2017-08-01 00:01:03,0,W 32 ST/7 AV,40.749405,-73.991020,JACKSON HTS NORTHERN - 81 via ROOSVLT,40.755322,-73.886139,NYCT_7424,40.749403,-73.990841,W 32 ST/AV OF THE AMERICAS,< 1 stop away,220.0,2017-08-01 00:01:37,24:01:11,00:01:37,NaT
B35,2017-08-01 00:00:52,0,39 ST/1 AV,40.656456,-74.012245,BROWNSVILLE M GASTON BL via CHURCH,40.656345,-73.907188,NYCT_406,40.651330,-73.938960,CHURCH AV/E 42 ST,approaching,107.0,2017-08-01 00:02:00,23:56:12,00:02:00,23:56:12
Q83,2017-08-01 00:01:18,1,227 ST/113 DR,40.702263,-73.730339,JAMAICA HILLSIDE - 153 via LIBERTY,40.706795,-73.804100,NYCT_6449,40.706532,-73.804177,153 ST/HILLSIDE AV,at stop,25.0,2017-08-01 00:01:27,24:00:00,00:01:27,NaT
M60-SBS,2017-08-01 00:01:05,0,BROADWAY/W 106 ST,40.801819,-73.967644,SELECT BUS SERVICE LA GUARDIA AIRPORT,40.768074,-73.862091,NYCT_5846,40.770403,-73.917687,HOYT AV/31 ST,4.1 miles away,6519.0,2017-08-01 00:06:47,23:39:14,00:06:47,23:39:14
M60-SBS,2017-08-01 00:01:05,0,BROADWAY/W 106 ST,40.801819,-73.967644,SELECT BUS SERVICE LA GUARDIA AIRPORT,40.768074,-73.862091,NYCT_5846,40.770403,-73.917687,HOYT AV/31 ST,4.1 miles away,6519.0,2017-08-01 00:06:47,23:44:32,00:06:47,23:44:32


Now let's drop observations whose ```DistanceFromStop``` is farther away than 30 ft.

In [7]:
mta = mta.loc[(mta['DistanceFromStop']<=30),]
print(mta.shape)

(1856274, 18)


Now let's drop missing ```ExpectedArrivalTime, expected_time & scheduled_time``` since we can't impute it at the moment.

In [8]:
mta = mta.loc[(mta['ExpectedArrivalTime'].notnull()),]
mta = mta.loc[(mta['expected_time'].notnull()),]
mta = mta.loc[(mta['scheduled_time'].notnull()),]
print(mta.shape)

(1315851, 18)


We'll create a new column that calculates the time difference between the expected and scheduled.

In [9]:
mta['time_diff'] = (pd.to_timedelta(mta['expected_time'].astype(str)) - 
                   pd.to_timedelta(mta['scheduled_time'].astype(str)))

In [10]:
mta['time_diff_mins'] = ((mta['time_diff'] / np.timedelta64(1, 'm')) + 1440).astype(int)
mta.dtypes

RecordedAtTime                        object
DirectionRef                           int64
OriginName                            object
OriginLat                            float64
OriginLong                           float64
DestinationName                       object
DestinationLat                       float64
DestinationLong                      float64
VehicleRef                            object
VehicleLocation.Latitude             float64
VehicleLocation.Longitude            float64
NextStopPointName                     object
ArrivalProximityText                  object
DistanceFromStop                     float64
ExpectedArrivalTime                   object
ScheduledArrivalTime                  object
expected_time                         object
scheduled_time                        object
time_diff                    timedelta64[ns]
time_diff_mins                         int32
dtype: object

## Visualizing the Data

Now we can visualize the time difference between expected and scheduled arrival times.

In [ ]:
# TODO

Looking at MTA API:

In [ ]:
# Key for Excel
mta_key = "51b681ab-bb14-4f29-9104-db15a7a41d41"